# Post OCR

When searching the Dagh Registers we are confronted with many different forms of the same words.
There are three causes for that

* OCR errors
* spelling variations
* morphology of the language

We deal with them separately.

## Techniques

To battle OCR errors, we work with letter 2,3-grams, filter them to retain
only the ones that may occur in real words, distill a set of legal words,
and then use an OCR key to search for corrections of illegal words.

Against spelling variations we use an anagram key to pick the best representant
among the variations in which a word occurs.

To deal with morphology, we use the same anagram hash, but with other settings.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import collections
import re

from pocr import PostOcr

## Read the corpus

We read the corpus from its text-fabric representation and make an inventory of all the letterwise n-grams,
n = 2,3.

In [3]:
P = PostOcr(4)
# P.charFreqs()

Loading TF data ...
done
Getting 2,3-grams for 229297 words in volume 4
Lower case characters: abcdefghijklmnopqrstuvwxyzèéêëïóöü
Upper case characters: ABCDEFGHIJKLMNOPQRSTUVWXYZÈÉËÏÓÖÜ
CHARACTERS (most frequent first):
enatodrslcigvhmyupwbkf'0jC12MGESP4DBx5JA3qHT687NOLz9UWVI(R^K*F.Qüéë):Z-Y/êèïËóöÜ[\«X"<>;!|£&?]$Ö•»ÈÓ}#~§É™°_Ï%€®
Call method charFreqs() to see the character frequencies
229297 words in 24717 distinct forms
Word forms with page/line numbers written to /Users/dirk/github/Dans-labs/clariah-dr/post/daghregister/004/forms.tsv
4189 distinct 2-grams
15647 distinct 3-grams


# OCR correction

First task: distil a lexicon of good words from the corpus.

Intuitition: make a list of the bi- and trigrams of letters, select the most freqent of these, and weed out the ones that 
cannot be part of words and are clearly ocr mistakes.

Then find all words in the corpus that consist of such bi- and trigrams.

We will miss rare words of which no correct form exists in the corpus.

We may try to correct such words by replacing their faulty bi- or trigrams by 
looking up legal words with the same *ocr-key*.

Let's inspect the amount of word froms the the grams occur in.
We take into account the frequencies of the words the n-grams occur in.

In [4]:
P.distFreq(2)


4189 2-grams

word freq│2-grams
─────────┼────────────────────────────────────────────────────────────────────────────────────
    47932│■1
    25107│■1
    20238│■1
    16314│■■2
    13151│■■2
    10601│■■■■4
     8545│■■■■■■6
     6888│■■■■■5
     5553│■■■■■■■■■■10
     4476│■■■■■■■■■■■■12
     3608│■■■■■■■■■■■■■■■■17
     2908│■■■■■■■■■■■■■■■■■■■23
     2345│■■■■■■■■■■■■■■■■■■■22
     1890│■■■■■■■■■■■■■■■■■■■■26
     1523│■■■■■■■■■■■■■■■■■■■■■■30
     1228│■■■■■■■■■■■■■■■■■■■■25
      990│■■■■■■■■■■■■■■■■■■■■■■31
      798│■■■■■■■■■■■■■■■■■■■■■■31
      643│■■■■■■■■■■■■■■■■■■■■■■■■■40
      519│■■■■■■■■■■■■■■■■■■■■■28
      418│■■■■■■■■■■■■■■■■■■■■■27
      337│■■■■■■■■■■■■■■■■■■■■■■31
      272│■■■■■■■■■■■■■■■■■■■■■■■■38
      219│■■■■■■■■■■■■■■■■■■■■■■■■■■■■54
      176│■■■■■■■■■■■■■■■■■■■■■■■■■41
      142│■■■■■■■■■■■■■■■■■■■■■■■■■■44
      115│■■■■■■■■■■■■■■■■■■■■■■■■38
       92│■■■■■■■■■■■■■■■■■■■■■■■■■■■48
       75│■■■■■■■■■■■■■■■■■■■■■■■■■■44
       60│■■■■■■■■■■■■■■■■■■■■

In [5]:
P.distFreq(3)


15647 3-grams

word freq│3-grams
─────────┼────────────────────────────────────────────────────────────────────────────────────
    12973│■1
     8882│■1
     7350│■1
     6081│■1
     5032│■1
     4164│■■■3
     3445│■■■■■■6
     2851│■■■■■■■■■9
     2359│■■■■■■6
     1952│■■■■■■■■■■■■■■17
     1615│■■■■■■■■■■■■■■■■■22
     1336│■■■■■■■■■■■■■■16
     1106│■■■■■■■■■■■■■■■■■■■29
      915│■■■■■■■■■■■■■■■■■■■■■■39
      757│■■■■■■■■■■■■■■■■■■■■■■42
      626│■■■■■■■■■■■■■■■■■■■■■■■■■■62
      518│■■■■■■■■■■■■■■■■■■■■■■■■■57
      429│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■90
      355│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■89
      294│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■87
      243│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■99
      201│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■105
      166│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■112
      138│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■106
      114│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■152
       94│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■170
       78│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■168
       65│■■■■■■■■■■■

Now for the word-starter grams:

In [6]:
P.distFreq(2, kind=False)


1600 2-grams word-starters only

word freq│2-grams
─────────┼───────────────────────────────────────────────────────────────────────────────────
    11225│■1
     7730│■■2
     6415│■1
     5323│■■2
     4417│■■■■4
     3666│■■■■■■6
     3042│■■■■4
     2524│■■■■4
     2095│■■2
     1738│■■■■■■■■8
     1443│■■■■■5
     1197│■■■■4
      993│■■■■■■■■■■■11
      824│■■■■■■■■■■■11
      684│■■■■■■6
      568│■■■■4
      471│■■■■■■■■■■■■■13
      391│■■■■■■■7
      324│■■■■■■■■■9
      269│■■■■■■■■■■■■■■■■■18
      223│■■■■■■■■■■■■■■■■■■■■■24
      185│■■■■■■■■■■■■■■14
      154│■■■■■■■■■■■■■■■■16
      128│■■■■■■■■■■■■■■■■17
      106│■■■■■■■■■■■■■■■■■■20
       88│■■■■■■■■■■■■■■■15
       73│■■■■■■■■■■■■■■■■■■20
       61│■■■■■■■■■■■■■■■■■■■■■25
       50│■■■■■■■■■■■■■■■■■■■21
       42│■■■■■■■■■■■■■■■■17
       35│■■■■■■■■■■■■■■■■17
       29│■■■■■■■■■■■■■■■■■■19
       24│■■■■■■■■■■■■■■■■■■■■22
       20│■■■■■■■■■■■■■■■■■■19
       16│■■■■■■■■■■■■■■■■■■■■■■■■32
       14│■■■■■■■■■■■■■■

In [7]:
P.distFreq(3, kind=False)


5229 3-grams word-starters only

word freq│3-grams
─────────┼────────────────────────────────────────────────────────────────────────────────────
     7738│■1
     5409│■1
     3780│■■2
     3161│■■2
     2642│■1
     2209│■■2
     1544│■■■■■■■■■9
     1291│■■■■■5
     1079│■■■■■■■■■■10
      902│■■■■4
      754│■■■■■■■■8
      631│■■■■■■■■■■■■■■16
      527│■■■■■■■■■■■■■■■18
      441│■■■■■■■■■■■■■■■17
      369│■■■■■■■■■■■■■■15
      308│■■■■■■■■■■■■■■■■■20
      258│■■■■■■■■■■■■■■■■■20
      215│■■■■■■■■■■■■■■■■■■■■■■35
      180│■■■■■■■■■■■■■■■■■■■■29
      151│■■■■■■■■■■■■■■■■■21
      126│■■■■■■■■■■■■■■■■■■■■■■34
      105│■■■■■■■■■■■■■■■■■■■■■■■39
       88│■■■■■■■■■■■■■■■■■■■■■■■■42
       74│■■■■■■■■■■■■■■■■■■■■■■■■■■49
       61│■■■■■■■■■■■■■■■■■■■■■■■■■■■55
       51│■■■■■■■■■■■■■■■■■■■■■■■■■■■54
       43│■■■■■■■■■■■■■■■■■■■■■■■■■■■58
       36│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■73
       30│■■■■■■■■■■■■■■■■■■■■■■■■■■■59
       25│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■91
       21│■■

And for the word-ender grams:

In [8]:
P.distFreq(2, kind=True)


1167 2-grams word-enders only

word freq│2-grams
─────────┼───────────────────────────────────────────────────────────────────────────────────
    47932│■1
    16314│■1
    10601│■1
     6888│■■2
     4476│■1
     3608│■■■3
     2908│■■■■4
     2345│■■■■■■6
     1890│■■■■■■■■8
     1523│■■■■■■■7
     1228│■■■■■■6
      990│■■2
      798│■■■■■■6
      643│■■■■■■■■■■■11
      519│■■■■■■■7
      418│■■2
      337│■■■■■■■7
      272│■■■■■■■■■9
      219│■■■■■■■■■■10
      176│■■■■■■■■■■■■■13
      142│■■■■■■■■■■■■■■14
      115│■■■■■■■■■■■■■■14
       92│■■■■■■■■■■■■■13
       75│■■■■■■■■■■■■■■■■■17
       60│■■■■■■■■■■■■■■14
       48│■■■■■■■■■■■11
       39│■■■■■■■■■■■■■■■■■17
       31│■■■■■■■■■■■■■■■15
       25│■■■■■■■■■■■■■■■■■■■20
       20│■■■■■■■■■■■■■■■■■■■20
       16│■■■■■■■■■■■■■■■■■■19
       13│■■■■■■■■■■■■■■■■■■■■■■■28
       11│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■49
        9│■■■■■■■■■■■■■■■■■■■■■■■■■31
        7│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■54
        6│■■■■■■■■■■■■■■■■■■■

In [9]:
P.distFreq(3, kind=True)


3874 3-grams word-enders only

word freq│3-grams
─────────┼────────────────────────────────────────────────────────────────────────────────────
    12973│■1
     8882│■1
     5032│■1
     4164│■1
     3445│■■■3
     2851│■■■3
     2359│■■■■4
     1952│■■■■■■6
     1615│■■■■■■■7
     1336│■■■■■■6
     1106│■■■■■■■7
      915│■■■■■■■■■9
      757│■■■■■■■7
      626│■■■■■■■■■■■■■■■15
      518│■■■■■■■7
      429│■■■■■■■■■■■■■■14
      355│■■■■■■■■■■■■■■■■■19
      294│■■■■■■■■■■■■■■■16
      243│■■■■■■■■■■10
      201│■■■■■■■■■■■■■■■■17
      166│■■■■■■■■■■■■■■■■17
      138│■■■■■■■■■■■■■■■■■■21
      114│■■■■■■■■■■■■■■■■■■■23
       94│■■■■■■■■■■■■■■■■■■21
       78│■■■■■■■■■■■■■■■■■■■■■■32
       65│■■■■■■■■■■■■■■■■■■■■■■■■37
       53│■■■■■■■■■■■■■■■■■■■■■■■35
       44│■■■■■■■■■■■■■■■■■■■■■■■■■42
       37│■■■■■■■■■■■■■■■■■■■■■■■■■■46
       30│■■■■■■■■■■■■■■■■■■■■■■■■■■46
       25│■■■■■■■■■■■■■■■■■■■■■■■■■■■49
       21│■■■■■■■■■■■■■■■■■■■■■■■■■■44
       17│■■■■■■■■■■■■■■■■■■■■■■■

# Legal grams

We try to weed out grams that cannot occur in real words.
A first approximation is to spot illegal consonant combinations.
We cannot do this for all the grams, but those that are at the start or end of words,
can easily be detected by inspection.

The first step is to show which consonant combinations occur.

We collect all 2-grams that consist of two consonants at the start of words.
Grams that only differ in case will be taken together.

Note that we single out a set of grams consisting of a space and then two consonants.
It is this set that we divide up into a legal subset and an illegal subset.

There are other grams, even other starter grams that we do not deal with in this step.

Above you see on each line the first letter of the combination with a `+` after it and its frequency above it.
Then the second letter of the combination follows, with the frequency of the combination above it.

In [10]:
P.getCombis(False, 2)

All 2-combis at the start of words

4526|1713|1670|399|373|245|61|28|10|10|4 |3 |2 |2 |2 |2 |1 |1 
s+  |t   |c   |p  |l  |w  |n |m |r |v |j |s |b |d |f |q |g |h 

2165|997|771|321|36|19|7 |3 |3 |3 |2 |1 |1 |1 
c+  |l  |h  |r  |b |k |n |f |p |t |c |j |q |s 

1535|864|510|135|7 |5 |3 |2 |2 |2 |1 |1 |1 |1 |1 
b+  |r  |l  |h  |c |b |s |g |j |t |f |k |m |n |p 

1488|1047|386|41|3 |2 |2 |2 |2 |1 |1 |1 
p+  |r   |l  |h |n |b |c |p |s |d |f |v 

1163|970|142|19|7 |6 |5 |5 |3 |3 |2 |1 
g+  |r  |h  |l |n |d |c |f |b |p |q |m 

1077|797|191|33|12|10|9 |8 |6 |4 |3 |2 |1 |1 
v+  |r  |l  |n |d |s |j |c |v |f |t |q |k |z 

903|346|330|160|25|11|9 |8 |4 |2 |2 |2 |2 |1 |1 
t+ |r  |w  |h  |s |b |l |v |c |j |m |n |t |f |g 

511|272|125|92|15|5 |2 
f+ |l  |r  |t |f |s |c 

291|250|10|9 |5 |5 |3 |2 |2 |1 |1 |1 |1 |1 
d+ |r  |v |t |c |w |p |m |s |b |d |j |k |l 

124|34|18|16|12|11|7 |5 |4 |3 |3 |3 |2 |2 |2 |1 |1 
h+ |s |g |w |v |c |f |m |n |d |h |p |b |j |r |l |q 

100|67|19|3 |3 |3 |2 |1 |1 |1 
w+ |t |r |d

We have inspected all possibilities, sometimes by looking up occurrences in the pdf file of the corpus.
That leads to a specification of which ones are the legal combinations.
All other combinations will be deemed illegal.

In [11]:
LEGAL_START_BI_COMBIS_SPEC = """
s +tcplwnmj
c +lhrn
b +rlh
p +rlh
g +rhln
v +rl
t +rwhs
f +lr
d +rw
h +
w +rh
k +lrhn
r +h
j +
m +
l +
n +
q +
z +
x +
"""

In [12]:
P.legalizeCombis(False, 2, LEGAL_START_BI_COMBIS_SPEC)

Legality declaration
+bh +bl +br
+ch +cl +cn +cr
+dr +dw
+fl +fr
+gh +gl +gn +gr
-h*
-j*
+kh +kl +kn +kr
-l*
-m*
-n*
+ph +pl +pr
-q*
+rh
+sc +sj +sl +sm +sn +sp +st +sw
+th +tr +ts +tw
+vl +vr
+wh +wr
-x*
-z*

Total number of start 2-grams:                1600
Total number of consonant combis among them:   288
Of which are   legal:                           87
         and illegal:                          201



Now the same for 3-grams:

In [13]:
P.getCombis(False, 3)

All 3-combis at the start of words

1707|464|299|265|244|204|164|54 |8  |2  |1  |1  |1  
st+ |a  |e  |o  |u  |r  |x  |i  |y  |n  |p  |t  |v  

1670|1617|42 |6  |2  |1  |1  |1  
sc+ |h   |b  |l  |e  |f  |o  |r  

1045|312|224|213|152|142|1  |1  
pr+ |e  |o  |i  |y  |a  |ó  |ö  

997|875|77 |34 |8  |1  |1  |1  
cl+|e  |a  |o  |i  |u  |è  |é  

970|800|70 |65 |26 |5  |3  |1  
gr+|o  |a  |i  |y  |e  |u  |r  

864|365|261|141|83 |13 |1  
br+|i  |e  |a  |o  |u  |f  

797|249|205|164|86 |59 |33 |1  
vr+|e  |u  |y  |o  |a  |i  |q  

771|397|123|105|79 |55 |9  |2  |1  
ch+|i  |a  |e  |r  |o  |y  |m  |x  

510|196|189|64 |55 |6  
bl+|y  |e  |a  |o  |i  

387|136|99 |63 |41 |29 |15 |3  |1  
sp+|a  |e  |r  |i  |o  |y  |d  |m  

386|356|13 |9  |7  |1  
pl+|a  |u  |e  |i  |o  

373|207|90 |32 |21 |8  |7  |6  |1  |1  
sl+|a  |e  |u  |o  |i  |x  |r  |c  |y  

346|160|80 |75 |25 |5  |1  
tr+|a  |e  |o  |i  |u  |y  

330|234|45 |43 |7  |1  
tw+|e  |y  |i  |a  |r  

321|93 |58 |53 |53 |48 |15 |1  
cr+|u 

In [14]:
LEGAL_START_TRI_COMBIS_SPEC = """
st -nptv
sc -blef
pr -óö
cl -èé
gr -r
br -f
vr -q
ch -mx
bl -
sp -dm
pl -
sl -xrc
tr -
tw -r
cr -n
fl -sgir
dr -
sw -
vl -
th -
gh +y
bh -i
fr -è
sn -clt
ph -h
sm -
ts +iu
gl -e
wr -
rh -
cn -i
gn -o
kl +e
kr -
dw -
kh -s
sj +o
kn -d
wh -
"""

In [15]:
P.legalizeCombis(False, 3, LEGAL_START_TRI_COMBIS_SPEC)

Legality declaration
-bhi
+bl*
-brf
-chm -chx
-clè -clé
-cni
-crn
+dr*
+dw*
-flg -fli -flr -fls
-frè
+ghy
-gle
-gno
-grr
-khs
+kle
-knd
+kr*
-phh
+pl*
-pró -prö
+rh*
-scb -sce -scf -scl
+sjo
-slc -slr -slx
+sm*
-snc -snl -snt
-spd -spm
-stn -stp -stt -stv
+sw*
+th*
+tr*
+tsi +tsu
-twr
+vl*
-vrq
+wh*
+wr*

Total number of start 3-grams:                5229
Total number of consonant combis among them:   312
Of which are   legal:                          257
         and illegal:                           55



Same exercition for combinations at the end of words.

In [16]:
P.getCombis(True, 2)

All 2-combis at the end of words

14725|3794|3173|2404|1509|890|775|708|648|299|220|122|91|71|14|5 |2 
+t   |n   |r   |h   |s   |d  |l  |f  |k  |m  |p  |c  |b |w |x |g |t 

8655|2398|1986|1973|1361|525|135|66|53|53|27|21|14|12|8 |8 |7 |5 |3 
+s  |r   |n   |l   |t   |h  |f  |k |m |p |b |c |v |g |d |w |s |x |j 

5142|5126|5 |4 |2 |2 |1 |1 |1 
+k  |c   |r |l |d |k |b |q |t 

5070|2624|2204|214|18|6 |1 |1 |1 |1 
+h  |c   |g   |t  |l |p |m |n |s |w 

905|689|166|39|9 |1 |1 
+x |c  |t  |k |l |r |s 

904|870|17|7 |5 |2 |1 |1 |1 
+f |f  |l |r |c |s |d |p |t 

742|670|30|17|12|4 |4 |1 |1 |1 |1 |1 
+p |m  |r |l |s |n |p |f |h |j |q |t 

706|199|162|117|110|44|18|15|13|12|9 |2 |2 |2 |1 
+j |r  |t  |l  |d  |n |b |v |k |s |g |f |h |p |m 

414|199|187|22|2 |2 |1 |1 
+d |r  |n  |l |g |s |c |w 

285|75|54|30|29|23|19|18|17|4 |4 |3 |3 |2 |2 |2 
+l |r |n |d |f |h |k |s |l |m |p |b |v |c |t |w 

194|149|17|14|3 |3 |2 |2 |1 |1 |1 |1 
+c |t  |n |d |k |l |b |s |g |h |r |w 

173|109|50|10|2 |2 
+b |g  |c |t 

In [17]:
LEGAL_END_BI_COMBIS_SPEC = """
t -t
s -b
k +crl
h -lmnsw
x -rs
f -csdpt
p -snfhjqt
j -vf
d -cw
l +r
c +tnkl
b +r
z +sx
q +c
n +rjh
m +rl
g -
r +spjm
v +lt
w +r
"""

In [18]:
P.legalizeCombis(True, 2, LEGAL_END_BI_COMBIS_SPEC)

Legality declaration
+br
+ck +cl +cn +ct
-dc -dw
-fc -fd -fp -fs -ft
+g*
-hl -hm -hn -hs -hw
-jf -jv
+kc +kl +kr
+lr
+ml +mr
+nh +nj +nr
-pf -ph -pj -pn -pq -ps -pt
+qc
+rj +rm +rp +rs
-sb
-tt
+vl +vt
+wr
-xr -xs
+zs +zx

Total number of end   2-grams:                1167
Total number of consonant combis among them:   229
Of which are   legal:                          131
         and illegal:                           98



In [19]:
P.getCombis(True, 3)

All 3-combis at the end of words

5121|1650|863|855|513|449|349|166|149|121|2  |2  |1  |1  
+ck |y   |o  |n  |i  |l  |e  |r  |u  |a  |d  |m  |b  |c  

3793|1643|1384|448|221|58 |32 |2  |1  |1  |1  |1  |1  
+nt |a   |e   |i  |o  |u  |y  |n  |h  |j  |m  |v  |ë  

3172|2606|455|71 |29 |4  |3  |3  |1  
+rt |e   |o  |a  |u  |é  |c  |y  |è  

2623|1184|807|211|183|82 |68 |42 |30 |10 |3  |1  |1  |1  
+ch |o   |i  |s  |a  |y  |e  |u  |r  |l  |p  |g  |n  |t  

2403|2202|201
+ht |c   |g  

2397|1647|564|155|17 |11 |1  |1  |1  
+rs |e   |o  |u  |a  |i  |c  |y  |ö  

2201|1368|463|104|103|62 |56 |22 |15 |3  |1  |1  |1  |1  |1  
+gh |n   |i  |a  |o  |y  |e  |r  |u  |l  |d  |h  |m  |p  |ü  

1986|1423|280|149|60 |51 |10 |9  |4  
+ns |e   |o  |a  |y  |i  |r  |u  |c  

1973|1657|237|22 |13 |13 |10 |9  |8  |3  |1  
+ls |a   |e  |r  |i  |o  |n  |y  |f  |u  |l  

1502|671|224|190|97 |80 |72 |70 |34 |33 |15 |5  |3  |2  |2  |1  |1  |1  |1  
+st |e  |a  |u  |r  |i  |n  |o  |h  |y  |t  |m  |g  |k  |l  |b  |d

In [20]:
LEGAL_END_TRI_COMBIS_SPEC = """
ck -dmbc
nt -nhmv
rt -écè
ch -p
ht -
rs - cö
gh -dhmpü
ns -c
ls -rnful
st -bdpü
ts -ft
dt -ó
ff -cë
lt -nfmé
cx -m
mp -bp
kt -
ft -cj
hs -e
mt -ócdl
pt -iu
th -dl
rd -c
rj -
nd -cl
tx -
tj -l
tc -
fs -i
ct -lt
lj -tls
dj -
sz -
bt -cg
cq -
rl +
ks -e
ms -oy
ps -
nj -
kx +
rm -
rp -
ng -y
wt -
ld -
rn -
cs -m
bj -
lf +
lp -
nc -ey
xt -
kj -
gs -f
sj -
lm -
gj -
lx -s
ds -dov
ws -
ph -o
vs -is
gt -
rk -
ss -a
xs -
lk -
rg +u
jn -st
js +i
kc -c
lc -
pp -
gd -n
hj -l
hn -
pj -
rb -
sd -
jr -
lv -
mj -
rf -
rw -
sr -v
tv -
xz -
"""

In [21]:
P.legalizeCombis(True, 3, LEGAL_END_TRI_COMBIS_SPEC)

Legality declaration
+bj*
-btc -btg
-chp
-ckb -ckc -ckd -ckm
+cq*
-csm
-ctl -ctt
-cxm
+dj*
-dsd -dso -dsv
-dtó
-ffc -ffë
-fsi
-ftc -ftj
-gdn
-ghd -ghh -ghm -ghp -ghü
+gj*
-gsf
+gt*
-hjl
+hn*
-hse
+ht*
-jns -jnt
+jr*
+jsi
-kcc
+kj*
-kse
+kt*
-kx*
+lc*
+ld*
-lf*
-ljl -ljs -ljt
+lk*
+lm*
+lp*
-lsf -lsl -lsn -lsr -lsu
-ltf -ltm -ltn -lté
+lv*
-lxs
+mj*
-mpb -mpp
-mso -msy
-mtc -mtd -mtl -mtó
-nce -ncy
-ndc -ndl
-ngy
+nj*
-nsc
-nth -ntm -ntn -ntv
-pho
+pj*
+pp*
+ps*
-pti -ptu
+rb*
-rdc
+rf*
+rgu
+rj*
+rk*
-rl*
+rm*
+rn*
+rp*
-rs  -rsc -rsö
-rtc -rtè -rté
+rw*
+sd*
+sj*
-srv
-ssa
-stb -std -stp -stü
+sz*
+tc*
-thd -thl
-tjl
-tsf -tst
+tv*
+tx*
-vsi -vss
+ws*
+wt*
+xs*
+xt*
+xz*

Total number of end   3-grams:                3874
Total number of consonant combis among them:   472
Of which are   legal:                          361
         and illegal:                          111



## Filter the set of n-grams

We leave out grams that

* are illegal because of the analysis of start- end end n-grams
* have a too low frequency (except when they are legal by the step above)
* have illegal characters in them
* have upper case letters preceded by lowercase letters

We might leave out legal grams in this process!

In [22]:
LIMIT = {
    2: 10,
    3: 10,
}

In [23]:
P.getLegals(LIMIT)


1170 legal 2-grams

occurrences│legal 2-gram
───────────┼──────────────────────────────────────────────────────────────────────────────────
      47932│■1
      25107│■1
      20238│■1
      16314│■■2
      13151│■■2
      10601│■■■■4
       8545│■■■■■■6
       6888│■■■■■5
       5553│■■■■■■■■■■10
       4476│■■■■■■■■■■■■12
       3608│■■■■■■■■■■■■■■■■■17
       2908│■■■■■■■■■■■■■■■■■■■■■■■23
       2345│■■■■■■■■■■■■■■■■■■■■■■22
       1890│■■■■■■■■■■■■■■■■■■■■■■■■■26
       1523│■■■■■■■■■■■■■■■■■■■■■■■■■■■■30
       1228│■■■■■■■■■■■■■■■■■■■■■■■■25
        990│■■■■■■■■■■■■■■■■■■■■■■■■■■■■31
        798│■■■■■■■■■■■■■■■■■■■■■■■■■■■■31
        643│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■39
        519│■■■■■■■■■■■■■■■■■■■■■■■■■■28
        418│■■■■■■■■■■■■■■■■■■■■■■■■■■27
        337│■■■■■■■■■■■■■■■■■■■■■■■■■■■■31
        272│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■37
        219│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■51
        176│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■40
        142│■■■■■■■■■■■■■■■■■■■■■■■■■

# Legal words

We now distil the words that are legal, by selecting the words whose bi- and trigrams are all legal.

In fact, we compute something slightly more general: for each word we compute its legality.

The legality of a word is the percentage of legal grams with respect to the total number of grams in it.

In [24]:
P.getLegality()

24655 word forms with legality distributed as:
legality│word form
────────┼─────────────────────────────────────────────────────────────────────────────────────
     100│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■13314
      91│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■2687
      83│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■3042
      76│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■1012
      69│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■1078
      63│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■445
      58│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■342
      52│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■238
      48│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■397
      44│■■■■■■■■■■■■■■■■■■■■■■46
      40│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■104
      36│■■■■■■■■■■■■■■17
      33│■■■■■■■■■■■■■■■■■■■■■■■■■■72
      30│■■■■■■■■■■■■■■■■■■■32
      28│■■■3
      25│■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■176
      21│■1
      19│■■■■■■■■■■■■14
      17│■■■■■

# OCR key

We compute the OCR keys of all forms, in order to see whether illegal words have counterparts with the same OCR key that are legal.
If so, we con choose the one with the minimum edit distance as a correction.

The idea of *ocr-key* I learned from Kai Niklas's
[Diplom Thesis](http://www.l3s.de/~tahmasebi/Diplomarbeit_Niklas.pdf).

In [25]:
P.getOcrKey("amw")

'o1i3v2'

Let's make an index of the word forms by ocr key.

In [26]:
P.makeOcrIndex()

24717 words clustered into 16827 ocr keys


# Correction

For every illegal word-like word we look to the legal words which share their ocr key with it.
Some words are just numbers, or transcripts of symbols.
We exclude them from the correctionprocess.
Our first criterion for word-like is: the word has at least twice as much non-digits as digits.

Of those words, we pick the ones that have the greatest similarity to the original word,
provided they have at least a certain threshold similarity to that word.
If there are several, we pick the word with the highest frequency.